In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

! cp "/content/gdrive/My Drive/InfoFaceRecognize/dataset/mydataset.zip" .
! unzip mydataset.zip

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  mydataset.zip
replace train/43.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


### Import Libs ###

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Flatten, 
                                     Dropout, ZeroPadding2D, Convolution2D,
                                     MaxPooling2D, Activation)
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model, Sequential
import matplotlib.pyplot as plt
from PIL import ImageFile
import pandas as pd
import numpy as np
import random

### Set Params ###

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
IMG_PATH = ""
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
BATCH_SIZE = 128
EPOCHS = 150

### Load and transform data ###

In [ ]:
def create_list(value, n_labels):
    cat_list = np.zeros(n_labels)
    cat_list[value] = 1
    return cat_list

In [ ]:
train_df = pd.read_csv("fairface_label_new_train.csv")
# idx = train_df[(train_df['race'] == 'East Asian') | (train_df['race'] == 'Southeast Asian')].index
# train_df.loc[idx, 'race'] = 'Asian'
train_df.head()

,file,age,gender,race,service_test
0,train/2.jpg,30-39,Female,Indian,False
1,train/3.jpg,3-9,Female,Black,False
2,train/4.jpg,20-29,Female,Indian,True
3,train/5.jpg,20-29,Female,Indian,True
4,train/7.jpg,40-49,Male,Middle Eastern,False


In [ ]:
train_df.age = pd.Categorical(train_df.age)
train_df['cat_age'] = train_df.age.cat.codes
train_df['cat_age'] = train_df.cat_age.apply(create_list, args=(train_df.age.nunique(),))

train_df.gender = pd.Categorical(train_df.gender)
train_df['cat_gender'] = train_df.gender.cat.codes
train_df['cat_gender'] = train_df.cat_gender.apply(create_list, args=(train_df.gender.nunique(),))

train_df.race = pd.Categorical(train_df.race)
train_df['cat_race'] = train_df.race.cat.codes
train_df['cat_race'] = train_df.cat_race.apply(create_list, args=(train_df.race.nunique(),))

print(train_df.shape)
train_df.head()

(54126, 8)


,file,age,gender,race,service_test,cat_age,cat_gender,cat_race
0,train/2.jpg,30-39,Female,Indian,False,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
1,train/3.jpg,3-9,Female,Black,False,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,train/4.jpg,20-29,Female,Indian,True,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
3,train/5.jpg,20-29,Female,Indian,True,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]"
4,train/7.jpg,40-49,Male,Middle Eastern,False,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"


In [ ]:
train_df, valid_df = train_test_split(train_df, test_size=.2, random_state=42, 
                                      stratify=train_df[['race', 'age', 'gender']])

In [ ]:
test_df = pd.read_csv("fairface_label_new_val.csv")
test_df.head()

,file,age,gender,race,service_test
0,val/1.jpg,3-9,Male,East Asian,False
1,val/2.jpg,50-59,Female,East Asian,True
2,val/4.jpg,20-29,Female,Latino_Hispanic,True
3,val/5.jpg,20-29,Male,Southeast Asian,False
4,val/6.jpg,30-39,Male,Latino_Hispanic,False


### Create and build model ###

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,3), 
                         include_top=False, weights='imagenet')

In [ ]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  154


In [ ]:
output_base_model = base_model.output

x = GlobalAveragePooling2D()(output_base_model)
x = Dense(1024, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(.2)(x)
race_output = Dense(7, activation='softmax', name='race')(x)

x = GlobalAveragePooling2D()(output_base_model)
x = Dense(1024, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(.2)(x)
gender_output = Dense(2, activation='softmax', name='gender')(x)

x = GlobalAveragePooling2D()(output_base_model)
x = Dense(1024, activation='relu')(x)
x = Dropout(.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(.2)(x)
age_output = Dense(9, activation='softmax', name='age')(x)

hybrid_model = Model(inputs=base_model.input, outputs=[race_output, 
                                                  gender_output, age_output])

In [ ]:
hybrid_model.compile(optimizer = Adam(), 
              loss = {
                  'race': 'categorical_crossentropy',
                  'gender': 'categorical_crossentropy',
                  'age': 'categorical_crossentropy'
                  
              }, 
              metrics = {
                  'race': 'accuracy',
                  'gender': 'accuracy',
                  'age': 'accuracy'
              }
             )

In [ ]:
hybrid_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

### Create generators ###

In [ ]:
train_datagen = ImageDataGenerator(
    horizontal_flip=True, 
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1. / 255
    )

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    IMG_PATH, 
    x_col='file',
    y_col=['cat_race', 'cat_gender', 'cat_age'],
    target_size=IMAGE_SIZE,
    class_mode="multi_output",
    batch_size=BATCH_SIZE,
    seed=42,
)

Found 43300 validated image filenames.


In [ ]:
validation_generator = validation_datagen.flow_from_dataframe(
    valid_df, 
    IMG_PATH, 
    x_col='file',
    y_col=['cat_race', 'cat_gender', 'cat_age'],
    target_size=IMAGE_SIZE,
    class_mode="multi_output",
    batch_size=BATCH_SIZE,
    seed=42,
)

Found 10826 validated image filenames.


In [ ]:
total_train = train_df.shape[0]
total_validate = valid_df.shape[0]

### Create callbacks ###

In [ ]:
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=1e-8)

mcp_save = ModelCheckpoint("/content/gdrive/My Drive/InfoFaceRecognize/weights/last_model_mnv2.h5", 
                           save_best_only=True, 
                           monitor='val_loss', mode='min', verbose=1)

callbacks = [earlystop, learning_rate_reduction, mcp_save]

### Train model ###

In [ ]:
history = hybrid_model.fit(
    train_generator, 
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=total_validate//BATCH_SIZE,
    steps_per_epoch=total_train//BATCH_SIZE,
    callbacks=callbacks
)

Epoch 1/150
338/338 [==============================] - 581s 2s/step - loss: 2.8320 - race_loss: 1.2274 - gender_loss: 0.3496 - age_loss: 1.2550 - race_accuracy: 0.5229 - gender_accuracy: 0.8328 - age_accuracy: 0.4738 - val_loss: 30.7557 - val_race_loss: 18.7244 - val_gender_loss: 7.5647 - val_age_loss: 4.4667 - val_race_accuracy: 0.1340 - val_gender_accuracy: 0.5112 - val_age_accuracy: 0.3329

Epoch 00001: val_loss improved from inf to 30.75573, saving model to /content/gdrive/My Drive/InfoFaceRecognize/weights/last_model_mnv2.h5


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/150
338/338 [==============================] - 525s 2s/step - loss: 2.4847 - race_loss: 1.0700 - gender_loss: 0.2853 - age_loss: 1.1293 - race_accuracy: 0.5844 - gender_accuracy: 0.8662 - age_accuracy: 0.5232 - val_loss: 24.7460 - val_race_loss: 14.4575 - val_gender_loss: 6.4654 - val_age_loss: 3.8231 - val_race_accuracy: 0.1561 - val_gender_accuracy: 0.5136 - val_age_accuracy: 0.2429

Epoch 00002: val_loss improved from 30.75573 to 24.74598, saving model to /content/gdrive/My Drive/InfoFaceRecognize/weights/last_model_mnv2.h5
Epoch 3/150
338/338 [==============================] - 524s 2s/step - loss: 2.3465 - race_loss: 1.0028 - gender_loss: 0.2610 - age_loss: 1.0827 - race_accuracy: 0.6143 - gender_accuracy: 0.8794 - age_accuracy: 0.5396 - val_loss: 11.4896 - val_race_loss: 3.2788 - val_gender_loss: 6.2875 - val_age_loss: 1.9233 - val_race_accuracy: 0.4202 - val_gender_accuracy: 0.5479 - val_age_accuracy: 0.4156

Epoch 00003: val_loss improved from 24.74598 to 11.48956, savin